### Постановка задачи
Загрузим подготовленные данные по энергопотреблению первых 20 зданий (building_id от 0 до 19).

Соберем два набора моделей: по дате (праздники, дни недели и т.д.) и по погоде.

Проведем 10 разбиений данных на обучающие/проверочные и выявим оптимальные веса моделей для каждого часа для каждого здания.

Вычислим оптимизированную метрику качества для ансамбля моделей.
        
* http://video.ittensive.com/machine-learning/ashrae/building_metadata.csv.gz
* http://video.ittensive.com/machine-learning/ashrae/weather_train.csv.gz
* http://video.ittensive.com/machine-learning/ashrae/train.0.csv.gz
Соревнование: https://www.kaggle.com/c/ashrae-energy-prediction/

© ITtensive, 2020

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [2]:
energy = pd.read_csv("energy.0-20.ready.csv.gz")

In [3]:
energy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175680 entries, 0 to 175679
Data columns (total 92 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   timestamp              175680 non-null  object 
 1   building_id            175680 non-null  int64  
 2   meter_reading          175680 non-null  float64
 3   primary_use            175680 non-null  object 
 4   air_temperature        175680 non-null  float64
 5   cloud_coverage         175680 non-null  float64
 6   dew_temperature        175680 non-null  float64
 7   precip_depth_1_hr      175680 non-null  float64
 8   sea_level_pressure     175680 non-null  float64
 9   wind_direction         175680 non-null  float64
 10  wind_speed             175680 non-null  float64
 11  air_temperature_diff1  175680 non-null  float64
 12  air_temperature_diff2  175680 non-null  float64
 13  hour                   175680 non-null  int64  
 14  weekday                175680 non-nu

In [4]:
lr_weather_colums = ['meter_reading_log','hour','building_id','air_temperature','dew_temperature',
                     'sea_level_pressure','wind_speed','air_temperature_diff1','air_temperature_diff2','cloud_coverage']

lr_days_columns = ['meter_reading_log','hour','building_id','is_holiday']
for wday in range(0,7):
    lr_days_columns.append('is_wday' + str(wday))
for week in range(1,54):
    lr_days_columns.append('is_w' + str(week))
for month in range(1,13):
    lr_days_columns.append('is_m' + str(month))

In [13]:
hours = range(0,24)
buildings = range(0,energy['building_id'].max() + 1)

In [14]:
def calculate_model(x,df_lr,lr_columns):
    lr = -1
    model = df_lr[x.building_id][x.hour]
    if len(model) > 0:
        lr = np.sum([x[c] * model[i] for i,c in enumerate(lr_columns[3:])])
        lr += model[len(lr_columns) - 3]
        lr = np.exp(lr)
    if lr < 0 or lr * lr == lr:
        lr = 0
    x['meter_reading_lr_q'] = (np.log(x.meter_reading + 1) - np.log(1 + lr)) ** 2
    return x

In [15]:
def train_model(df,columns):
    df_train_lr = pd.DataFrame(df,columns=columns)
    df_lr = [[]] * len(buildings)
    for building in buildings:
        df_lr[building] = [[]] * len(hours)
        df_train_b = df_train_lr[df_train_lr['building_id'] == building]
        for hour in hours:
            df_train_bh = df_train_b[df_train_b['hour'] == hour]
            y = df_train_bh['meter_reading_log']
            x = df_train_bh.drop(['meter_reading_log','hour','building_id'],axis=1)
            model = LinearRegression(fit_intercept=False).fit(x,y)
            df_lr[building][hour] = model.coef_
            df_lr[building][hour] = np.append(df_lr[building][hour],model.intercept_)
    return df_lr

def calculate_weights_model(df_test,df_train,lr_columns):
    df_test = df_test.apply(calculate_model,axis=1,result_type='expand',
                           df_lr=train_model(df_train,lr_columns),
                           lr_columns=lr_columns)
    return pd.Series(df_test.groupby(['hour','building_id']).sum()['meter_reading_lr_q'])
def calculate_weights():
    df_train,df_test = train_test_split(energy[energy['meter_reading'] > 0],test_size=0.2)
    return (calculate_weights_model(df_test,df_train,lr_weather_colums),
           calculate_weights_model(df_test,df_train,lr_days_columns))

In [16]:
weights_weather = []
weights_days = []
for i in range(0,10):
    print(i)
    weights_weather_model,weights_days_model = calculate_weights()
    if len(weights_weather) > 0:
        weights_weather += weights_weather_model
    else:
        weights_weather = weights_weather_model
    if len(weights_days) > 0:
        weights_days += weights_days_model
    else:
        weights_days = weights_days_model
        
weights = [0] * len(buildings)
for b in buildings:
    
    weights[b] = [0] * len(hours)
    for h in hours:
        if weights_weather.loc[h].at[b] > weights_days.loc[h].at[b]:
            weights[b][h] = 1
print(weights)

0


<ipython-input-14-f7b5083fc8a3>:7: RuntimeWarning: overflow encountered in exp
  lr = np.exp(lr)


1


<ipython-input-14-f7b5083fc8a3>:7: RuntimeWarning: overflow encountered in exp
  lr = np.exp(lr)


2


...

8


<ipython-input-14-f7b5083fc8a3>:7: RuntimeWarning: overflow encountered in exp
  lr = np.exp(lr)


9


<ipython-input-14-f7b5083fc8a3>:7: RuntimeWarning: overflow encountered in exp
  lr = np.exp(lr)


[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [17]:
from sklearn.model_selection import train_test_split

In [18]:
energy_train,energy_test = train_test_split(energy[energy['meter_reading'] > 0], test_size=0.2)

In [22]:
energy_lr_days = train_model(energy_train,lr_days_columns)
energy_lr_weather = train_model(energy_train,lr_weather_colums)

In [27]:
def calculate_model_ensemble(x,model,columns):
    lr = -1
    if len(model) > 0:
        lr = np.sum([x[c] * model[i] for i,c in enumerate(columns[3:])])
        lr += model[len(columns) - 3]
        lr = np.exp(lr)
    if lr < 0 or lr*lr == lr:
        lr = 0
    return lr
def calculate_models_ensemble(x):
    lr_d = calculate_model_ensemble(x,energy_lr_days[x.building_id][x.hour],lr_days_columns)
    lr_w = calculate_model_ensemble(x,energy_lr_days[x.building_id][x.hour],lr_weather_colums)
    if weights[x.building_id][x.hour] == 1:
        lr = lr_d
    else:
        lr = lr_w
    lr_sum = (lr_w + lr_d) / 2
    x['meter_reading_lr_q'] = (np.log(x.meter_reading + 1) + np.log(1 + lr)) ** 2
    x['meter_reading_sum_q'] = (np.log(x.meter_reading + 1) + np.log(1 + lr_sum)) ** 2
    return x

В теории, в идеальном случае, ансамбль линейной регрессии не должен давать никакого преимущества, потому что если
\begin{equation}
z_1 = Ax + By + C, z_2 = Ds + Et + F, то
\end{equation}
\begin{equation}
z = \alpha z_1 + \beta z_2 = \alpha Ax + \alpha By + \alpha C + \beta Ds + \beta Et + \beta F = A_1x + B_1y + D_1s + E_1t + F_1
\end{equation}
И по сути ансамбль линейной регрессии - это просто линейная регрессия по всем переменным. Но при использовании небольших наборов (чтобы исключить переобучение) связанных переменных для разных моделей регрессии можно получить небольшой выигрыш.

Ансамбль регрессии в нашем случае не дает никакого улучшения относительно регрессии по совокупному набору параметров.

Однако, использование усредненной суммы показателей каждой конкретной модели дало выигрыш порядка 6% относительно модели по всем показателям. В этом случае сумму моделей линейной регрессии "компенсирует" ошибки каждой конкретной модели и работает точнее.

In [28]:
energy_test = energy_test.apply(calculate_models_ensemble,axis=1,result_type='expand')

print(np.sqrt(energy_test['meter_reading_lr_q'].sum() / len(energy_test)))
print(np.sqrt(energy_test['meter_reading_sum_q'].sum() / len(energy_test)))

<ipython-input-27-f183ec8eab26>:6: RuntimeWarning: overflow encountered in exp
  lr = np.exp(lr)


5.8755614566891925
10.815833230914512
